<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper

This Project uses the Original [Whisper](https://github.com/openai/whisper) Project instead of Decipher Project.

For Everyone who want to use the Original Whisper Project

# Usage

1. Check the Runtime Type is on GPU Mode in ``Runtime -> Change Runtime Type ``
1. Change the Settings at Settings Code Block
1. Click ``Runtime -> Run all`` (CTRL+F9)
1. Click on the Connect to Google Drive
1. And Wait for a moment your Generated Srt should be on ``Google Drive -> Whisper -> result``

## Add Google Drive Access
(at Google Drive/Whisper)

In [1]:
#@markdown # **Download Opition**
disable_google_drive = "no" #@param ['yes', 'no']
download_method = "bashupload" #@param ['bashupload', 'direct']
#@markdown The Driect Method will **NOT** Working if the ``shutdown_after_complete`` is enabled. It will **Fall Back to bashupload** instead.
#@markdown # Disclaimer : The Direct Download sometimes doesn't work on Firefox Type Web Browser. </p> Please **use anonfile or transfer.sh instead**
if disable_google_drive.lower() == 'no':

  from google.colab import drive
  import os

  path = '/content/gdrive/MyDrive/Whisper/'
  path2 = '/content/gdrive/MyDrive/Whisper/result'
  drive.mount('/content/gdrive')

  if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path2)

# Calcuate the Execution Time.
from datetime import datetime
start_time = datetime.now()

# Install Components

Package : openai-whisper yt-dlp ipython pyperclip wget pycurl

In [2]:
! apt-get install ffmpeg
! pip install openai-whisper yt-dlp ipython pyperclip wget pycurl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# GPU Checkup

If Nothing shows up means you didn't enable GPU in the ``Runtime -> Change Runtime type``

In [3]:
! nvidia-smi

Sun May 26 17:41:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Settings


In [4]:
import os
#@markdown File Format **must be mp3**
upload_own_audio_file = "no" #@param ['yes', 'no']
#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=5MbHx2GHe7U" #@param {type:"string"}
# File Name Change
new_filename = "Scaring Scammers to Shutdown" #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
language = "" #@param {type:"string"}
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **Other Options**
Generate_Plain_Document = "yes" #@param ['yes','no']
shutdown_after_complete = "yes" #@param ['yes','no']
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
enable_cookies = "no" #@param ['yes', 'no']

# If Enable Cookies is Enabled
Upload your Cookies file in here

In [5]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# Download the Video or upload the audio
using the yt-dlp and ffmpeg

In [6]:
from google.colab import files
import shutil
import yt_dlp

if upload_own_audio_file == "yes":
  if not os.path.isfile('audio.mp3'):
      own_audio = files.upload()
      if not os.path.exists('content'):
        os.mkdir('content')
        for filename in own_audio.keys():
            os.rename(filename, 'audio.mp3')
            os.replace('audio.mp3', 'content/audio.mp3')

else:
    if enable_cookies.lower() == "yes":
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
            'cookiefile': 'cookies.txt'
        }
    else:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': 'content/audio',
        }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YouTube_Video_Link])


Saving Pranking Scammers ft. Mark Rober [0dT6jB2Dbmk].mp3 to Pranking Scammers ft. Mark Rober [0dT6jB2Dbmk].mp3


# Generating the SRT file

In [7]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"
if language.lower() == '':
  result = model.transcribe(input_file, verbose=True, task=task)
else:
  result = model.transcribe(input_file, verbose=True, task=task, language=language)

if disable_google_drive.lower() == 'no':

  transcription_root = "/content/gdrive/MyDrive/Whisper/result"

else:
  os.mkdir("result")
  transcription_root = "/content/result"

# Setup Options for SRT/TXT file
options = {
    'max_line_width': None,
    'max_line_count': None,
    'highlight_words': False
}

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename, options)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2, options)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|█████████████████████████████████████| 1.42G/1.42G [00:32<00:00, 47.2MiB/s]


Whisper model loaded.
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:03.360]  If you found out that a scam call centre had CCTV,
[00:03.360 --> 00:07.840]  wouldn't it be great to bring some pranks directly to their doorstep?
[00:30.000 --> 00:49.280]  I think everyone who's had a scam phone call will enjoy this one.
[00:49.280 --> 01:02.560]  There is an area which is known as the telephone scam capital of the world,
[01:02.560 --> 01:11.040]  and that area is sector 5 of Kolkata in India. And although the scam call centres were located
[01:11.040 --> 01:15.200]  there, it would be Indian people who would be key to try to bring them down.
[01:16.160 --> 01:20.240]  I'm calling you from the headquarters of Microsoft Corporation. We are located in Redmond City,
[01:20.240 --> 01:25.920]  Washington DC, United States. My name is Richard Anderson. I'm working here as a senior SDE
[01:25.920 --> 01:

# If Google Drive Gets Disabled




In [10]:
import shutil
import os.path
import requests
import pycurl
from io import BytesIO

if disable_google_drive.lower() == 'yes':
    # Creating the ZIP file
    archived = shutil.make_archive(new_filename, 'zip', '/content/result')

    if os.path.exists(archived):
        print(archived)

        if shutdown_after_complete.lower() == "yes":
          print("Because you can't use direct download with shutdown after complete. So we decided to switch to bashupload insead.")
          file_path = archived
          upload_url = 'bashupload.com'
          new_new_filename = f'{new_filename}.zip'

          # Initialize a cURL object
          c  = pycurl.Curl()

          # Prepare the form data
          c.setopt(c.URL, upload_url)
          c.setopt(c.HTTPPOST, [
              ('file', (
                # Set the filename in the form data
                  c.FORM_FILE, file_path,
                  c.FORM_FILENAME, new_new_filename
                )),
            ])

          # Capture the response
          response = BytesIO()
          c.setopt(c.WRITEFUNCTION, response.write)

          # Perform the file upload
          c.perform()
          data = response.getvalue().decode("UTF-8")
          print(data)

        elif download_method.lower() == "bashupload":
          file_path = archived
          upload_url = 'bashupload.com'
          new_new_filename = f'{new_filename}.zip'

          # Initialize a cURL object
          c  = pycurl.Curl()

          # Prepare the form data
          c.setopt(c.URL, upload_url)
          c.setopt(c.HTTPPOST, [
              ('file', (
                # Set the filename in the form data
                  c.FORM_FILE, file_path,
                  c.FORM_FILENAME, new_new_filename
                )),
            ])

          # Capture the response
          response = BytesIO()
          c.setopt(c.WRITEFUNCTION, response.write)

          # Perform the file upload
          c.perform()
          data = response.getvalue().decode("UTF-8")
          print(data)

        else:
            files.download(archived)
            print("File downloaded successfully.")

    else:
        print("ZIP file not created")

/content/Scaring Scammers to Shutdown.srt.zip
Because you can't use direct download with shutdown after complete. So we decided to switch to bashupload insead.


Uploaded 1 file, 27 629 bytes

wget http://bashupload.com/aibX8/EMFmL.zip





# Auto Shutdown the Runtime

In [11]:
import time
time.sleep (2)
end_time = datetime.now()
print('Execution Time: {}'.format(end_time - start_time))

if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()


Execution Time: 0:13:08.305628
Shutting Down the Runtime Because Shutdown After Complete is on
